In [1]:
import sys

sys.path.insert(0, "../")

import pandas as pd
from mdu.eval.table_analysis_utils import (
    transform_by_tasks,
    select_composite_and_components,
    check_composite_dominance,
    compute_average_ranks,
    analyze_composite_pareto_performance,
)
from notebooks.table_utils import with_avg_row, mean_pm_std, fmt_valvar
from mdu.unc.constants import OTTarget, ScalingType

# Set pandas display options to show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

%load_ext autoreload
%autoreload 2

In [2]:
target_distr = OTTarget.BETA.value
scaling_type = ScalingType.FEATURE_WISE.value
grid_size = 5
n_targets_multiplier = 1


df = pd.read_csv(
    f"../resources/refactored/benchmark_entropic_target_{target_distr}_eps_0.5_scaling_type_{scaling_type}_iters_1000_tol_1e-06_rs_42_grid_size_{grid_size}_n_targets_multiplier_{n_targets_multiplier}.csv"
)

In [3]:
df.head(10)

,ind_dataset,ood_dataset,measure,uncertainty_type,gname,risk_type,gt_approximation,pred_approximation,ensemble_group,problem_type,roc_auc,average_precision,accuracy,aurc,acc_cov_auc,coverage_at_1pct_error,coverage_at_2pct_error,coverage_at_5pct_error,n_ind_samples,n_ood_samples,n_correct,n_incorrect,ensemble_accuracy
0,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,ood_detection,0.912015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963750
1,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,misclassification_detection,0.944999,0.381728,0.963750,NaN,NaN,NaN,NaN,NaN,7200,NaN,6939.0,261.0,0.963750
2,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,selective_prediction,NaN,NaN,0.963750,0.002928,0.996933,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963750
3,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,ood_detection,0.910690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964306
4,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,misclassification_detection,0.944610,0.374033,0.964306,NaN,NaN,NaN,NaN,NaN,7200,NaN,6943.0,257.0,0.964306
5,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,selective_prediction,NaN,NaN,0.964306,0.002930,0.996931,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.964306
6,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,ood_detection,0.911888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963333
7,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,misclassification_detection,0.943641,0.360288,0.963333,NaN,NaN,NaN,NaN,NaN,7200,NaN,6936.0,264.0,0.963333
8,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,selective_prediction,NaN,NaN,0.963333,0.003023,0.996838,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963333
9,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,3,ood_detection,0.911436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964167


In [4]:
print(df.ind_dataset.unique())
print(df.ood_dataset.unique())

['cifar10' 'cifar100' 'tiny_imagenet']
['cifar100' 'cifar10' 'tiny_imagenet' 'svhn' 'imagenet_a' 'imagenet_o'
 'imagenet_r']


In [5]:
include_std = True

if not include_std:
    transformed_df = transform_by_tasks(df, include_std=include_std)
else:
    transformed_df, std_df = transform_by_tasks(df, include_std=include_std)

In [6]:
transformed_df

measure                                  R_b 1 (Brier)  R_b 1 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.942466          0.942267   
              cifar10 [selective]             0.996836          0.996818   
              cifar100 [ood]                  0.914436          0.916906   
              svhn [ood]                      0.958357          0.962992   
              tiny_imagenet [ood]             0.908611          0.911364   
cifar100      cifar10 [ood]                   0.773000          0.773270   
              cifar100 [miscls]               0.857773          0.845094   
              cifar100 [selective]            0.920203          0.915943   
              svhn [ood]                      0.858293          0.870120   
              tiny_imagenet [ood]             0.789521          0.809888   
tiny_imagenet tiny_imagenet [miscls]          0.854657          0.844739   
              tiny_imagenet [selective]       0.892676          0.888853   
              imagenet_a [ood]                0.827188          0.835350   
              imagenet_r [ood]                0.816241          0.825339   
              imagenet_o [ood]                0.720970          0.724312   

measure                                  R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.942228          0.941776   
              cifar10 [selective]                 0.996827          0.996810   
              cifar100 [ood]                      0.914506          0.913191   
              svhn [ood]                          0.958530          0.956294   
              tiny_imagenet [ood]                 0.908679          0.907210   
cifar100      cifar10 [ood]                       0.773388          0.772177   
              cifar100 [miscls]                   0.856181          0.858986   
              cifar100 [selective]                0.919719          0.920576   
              svhn [ood]                          0.860970          0.855937   
              tiny_imagenet [ood]                 0.806267          0.803022   
tiny_imagenet tiny_imagenet [miscls]              0.853150          0.854657   
              tiny_imagenet [selective]           0.892115          0.892648   
              imagenet_a [ood]                    0.830074          0.826338   
              imagenet_r [ood]                    0.819225          0.815484   
              imagenet_o [ood]                    0.722622          0.721199   

measure                                  R_b 2 (Brier)  R_b 2 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.945782          0.944886   
              cifar10 [selective]             0.996959          0.996920   
              cifar100 [ood]                  0.913049          0.916237   
              svhn [ood]                      0.957686          0.963150   
              tiny_imagenet [ood]             0.905817          0.909549   
cifar100      cifar10 [ood]                   0.774385          0.774538   
              cifar100 [miscls]               0.865555          0.850089   
              cifar100 [selective]            0.922462          0.917541   
              svhn [ood]                      0.856738          0.870178   
              tiny_imagenet [ood]             0.937085          0.926019   
tiny_imagenet tiny_imagenet [miscls]          0.863376          0.849698   
              tiny_imagenet [selective]       0.895661          0.890666   
              imagenet_a [ood]                0.834463          0.840955   
              imagenet_r [ood]                0.824133          0.830950   
              imagenet_o [ood]                0.735803          0.734891   

measure                                  R_b 2 (Spher

In [7]:
std_df

R_b 1 (Brier)  R_b 1 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.001560          0.001585   
              cifar10 [selective]             0.000044          0.000049   
              cifar100 [ood]                  0.001158          0.001002   
              svhn [ood]                      0.002077          0.002252   
              tiny_imagenet [ood]             0.001202          0.001037   
cifar100      cifar10 [ood]                   0.002073          0.002164   
              cifar100 [miscls]               0.002891          0.002761   
              cifar100 [selective]            0.000760          0.000890   
              svhn [ood]                      0.005678          0.006667   
              tiny_imagenet [ood]             0.000841          0.000847   
tiny_imagenet tiny_imagenet [miscls]          0.003170          0.002899   
              tiny_imagenet [selective]       0.000667          0.000570   
              imagenet_a [ood]                0.002875          0.002358   
              imagenet_r [ood]                0.003388          0.003457   
              imagenet_o [ood]                0.004394          0.003811   

                                         R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.001625          0.001838   
              cifar10 [selective]                 0.000046          0.000052   
              cifar100 [ood]                      0.001188          0.001341   
              svhn [ood]                          0.001807          0.001607   
              tiny_imagenet [ood]                 0.001218          0.001339   
cifar100      cifar10 [ood]                       0.001835          0.001594   
              cifar100 [miscls]                   0.002897          0.002921   
              cifar100 [selective]                0.000799          0.000806   
              svhn [ood]                          0.006101          0.005910   
              tiny_imagenet [ood]                 0.000601          0.000480   
tiny_imagenet tiny_imagenet [miscls]              0.003050          0.003052   
              tiny_imagenet [selective]           0.000617          0.000647   
              imagenet_a [ood]                    0.002685          0.002844   
              imagenet_r [ood]                    0.003200          0.003028   
              imagenet_o [ood]                    0.004105          0.004197   

                                         R_b 2 (Brier)  R_b 2 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.002001          0.001597   
              cifar10 [selective]             0.000064          0.000054   
              cifar100 [ood]                  0.000880          0.000814   
              svhn [ood]                      0.003653          0.002954   
              tiny_imagenet [ood]             0.001008          0.000887   
cifar100      cifar10 [ood]                   0.001566          0.001926   
              cifar100 [miscls]               0.002871          0.002655   
              cifar100 [selective]            0.000728          0.000870   
              svhn [ood]                      0.005974          0.006407   
              tiny_imagenet [ood]             0.001409          0.000946   
tiny_imagenet tiny_imagenet [miscls]          0.003091          0.002722   
              tiny_imagenet [selective]       0.000509          0.000445   
              imagenet_a [ood]                0.002697          0.002300   
              imagenet_r [ood]                0.002514          0.002893   
              imagenet_o [ood]                0.003341          0.003106   

                                         R_b 2 (Spherical)  R_b 2 (Zero-one)  \
ind_dataset   

In [8]:
from configs.interesting_compositions import INTERESTING_COMPOSITIONS

for k in INTERESTING_COMPOSITIONS.keys():
    print(k)

COMPOSITE BAYES ALL OUTER
COMPOSITE BAYES ALL INNER
COMPOSITE BAYES ALL CENTRAL
COMPOSITE BAYES ALL OUTER + M
COMPOSITE BAYES ALL INNER + M
COMPOSITE BAYES ALL CENTRAL + M
COMPOSITE EXCESS ALL OUTER OUTER
COMPOSITE EXCESS ALL OUTER INNER
COMPOSITE EXCESS ALL OUTER CENTRAL
COMPOSITE EXCESS ALL INNER OUTER
COMPOSITE EXCESS ALL INNER CENTRAL
COMPOSITE EXCESS ALL CENTRAL OUTER
COMPOSITE EXCESS ALL CENTRAL INNER
COMPOSITE EXCESS ALL OUTER OUTER + M
COMPOSITE EXCESS ALL OUTER INNER + M
COMPOSITE EXCESS ALL OUTER CENTRAL + M
COMPOSITE EXCESS ALL INNER OUTER + M
COMPOSITE EXCESS ALL INNER CENTRAL + M
COMPOSITE EXCESS ALL CENTRAL OUTER + M
COMPOSITE EXCESS ALL CENTRAL INNER + M
COMPOSITE EAT LOGSCORE OUTER OUTER
COMPOSITE EAT LOGSCORE OUTER INNER
COMPOSITE EAT LOGSCORE OUTER CENTRAL
COMPOSITE EAT LOGSCORE INNER OUTER
COMPOSITE EAT LOGSCORE INNER CENTRAL
COMPOSITE EAT LOGSCORE CENTRAL OUTER
COMPOSITE EAT LOGSCORE CENTRAL INNER
COMPOSITE EAT LOGSCORE OUTER OUTER + M
COMPOSITE EAT LOGSCORE OUTER I

In [9]:
measure_name = "COMPOSITE EAT LOGSCORE OUTER OUTER + M"
res_df = select_composite_and_components(transformed_df, measure_name)
res_df_std = select_composite_and_components(std_df, measure_name)

res_df_with_dominance = check_composite_dominance(res_df)

composite_pareto_results = analyze_composite_pareto_performance(
    transformed_df, INTERESTING_COMPOSITIONS, do_for_each_measure=True, different_only=False
)

In [10]:
composite_pareto_results

{'COMPOSITE BAYES ALL OUTER': {'pareto_count': 36,
  'total_problems': 105,
  'pareto_percentage': 34.285714285714285,
  'average_pareto_depth': np.float64(0.8476190476190476),
  'median_pareto_depth': np.float64(1.0),
  'individual_measures': {'R_b 1 (Logscore)': {'pareto_count': 85,
    'total_problems': 105,
    'pareto_percentage': 80.95238095238095,
    'average_pareto_depth': np.float64(0.44761904761904764),
    'median_pareto_depth': np.float64(0.0)},
   'R_b 1 (Brier)': {'pareto_count': 64,
    'total_problems': 105,
    'pareto_percentage': 60.952380952380956,
    'average_pareto_depth': np.float64(0.8285714285714286),
    'median_pareto_depth': np.float64(0.0)},
   'R_b 1 (Spherical)': {'pareto_count': 54,
    'total_problems': 105,
    'pareto_percentage': 51.42857142857142,
    'average_pareto_depth': np.float64(0.6476190476190476),
    'median_pareto_depth': np.float64(0.0)},
   'R_b 1 (Zero-one)': {'pareto_count': 31,
    'total_problems': 105,
    'pareto_percentage': 29

In [11]:
display(res_df_with_dominance)
display(res_df_std)

print("==" * 100)
print("Pareto Percentages:")

for k, result in composite_pareto_results.items():
    if measure_name == k:
        indv_ = result['individual_measures']
        print(measure_name, result['pareto_percentage'])
        for name, el in indv_.items():
            print(name, el['pareto_percentage'])

measure                                  R_e 1 1 (Logscore)  \
ind_dataset   eval                                            
cifar10       cifar10 [miscls]                     0.940430   
              cifar10 [selective]                  0.996843   
              cifar100 [ood]                       0.904730   
              svhn [ood]                           0.942616   
              tiny_imagenet [ood]                  0.895731   
cifar100      cifar10 [ood]                        0.724548   
              cifar100 [miscls]                    0.818006   
              cifar100 [selective]                 0.909667   
              svhn [ood]                           0.755848   
              tiny_imagenet [ood]                  0.999852   
tiny_imagenet tiny_imagenet [miscls]               0.813053   
              tiny_imagenet [selective]            0.879249   
              imagenet_a [ood]                     0.781130   
              imagenet_r [ood]                     0.774366   
              imagenet_o [ood]                     0.752832   

measure                                  R_t 1 1 (Logscore)  R_b 1 (Logscore)  \
ind_dataset   eval                                                              
cifar10       cifar10 [miscls]                     0.943244          0.942267   
              cifar10 [selective]                  0.996864          0.996818   
              cifar100 [ood]                       0.911507          0.916906   
              svhn [ood]                           0.956506          0.962992   
              tiny_imagenet [ood]                  0.903615          0.911364   
cifar100      cifar10 [ood]                        0.774023          0.773270   
              cifar100 [miscls]                    0.853064          0.845094   
              cifar100 [selective]                 0.918425          0.915943   
              svhn [ood]                           0.867727          0.870120   
              tiny_imagenet [ood]                  0.999997          0.809888   
tiny_imagenet tiny_imagenet [miscls]               0.850815          0.844739   
              tiny_imagenet [selective]            0.890910          0.888853   
              imagenet_a [ood]                     0.846305          0.835350   
              imagenet_r [ood]                     0.836572          0.825339   
              imagenet_o [ood]                     0.753524          0.724312   

measure                                  mahalanobis  \
ind_dataset   eval                                     
cifar10       cifar10 [miscls]              0.927621   
              cifar10 [selective]           0.996266   
              cifar100 [ood]                0.912238   
              svhn [ood]                    0.934311   
              tiny_imagenet [ood]           0.910273   
cifar100      cifar10 [ood]                 0.534822   
              cifar100 [miscls]             0.573908   
              cifar100 [selective]          0.810550   
              svhn [ood]                    0.678832   
              tiny_imagenet [ood]           0.622941   
tiny_imagenet tiny_imagenet [miscls]        0.416754   
              tiny_imagenet [selective]     0.659777   
              imagenet_a [ood]              0.440974   
              imagenet_r [ood]              0.404755   
              imagenet_o [ood]              0.512686   

measure                                  composite eat logscore outer outer + m  \
ind_dataset   eval                                                                
cifar10       cifar10 [miscls]                                         0.943972   
              cifar10 [selective]                                      0.996923   
              cifar100 [ood]                                           0.917981   
              svhn [ood]                                               0.957479   
              tiny_imagenet [ood]                                      0.912092   
cifar100      ci

R_e 1 1 (Logscore)  \
ind_dataset   eval                                            
cifar10       cifar10 [miscls]                     0.003253   
              cifar10 [selective]                  0.000145   
              cifar100 [ood]                       0.000337   
              svhn [ood]                           0.012131   
              tiny_imagenet [ood]                  0.000245   
cifar100      cifar10 [ood]                        0.001497   
              cifar100 [miscls]                    0.002878   
              cifar100 [selective]                 0.001027   
              svhn [ood]                           0.013424   
              tiny_imagenet [ood]                  0.000038   
tiny_imagenet tiny_imagenet [miscls]               0.001207   
              tiny_imagenet [selective]            0.001284   
              imagenet_a [ood]                     0.003696   
              imagenet_r [ood]                     0.003244   
              imagenet_o [ood]                     0.002095   

                                         R_t 1 1 (Logscore)  R_b 1 (Logscore)  \
ind_dataset   eval                                                              
cifar10       cifar10 [miscls]                     0.002413          0.001585   
              cifar10 [selective]                  0.000086          0.000049   
              cifar100 [ood]                       0.000599          0.001002   
              svhn [ood]                           0.007015          0.002252   
              tiny_imagenet [ood]                  0.000583          0.001037   
cifar100      cifar10 [ood]                        0.001598          0.002164   
              cifar100 [miscls]                    0.002791          0.002761   
              cifar100 [selective]                 0.000848          0.000890   
              svhn [ood]                           0.005873          0.006667   
              tiny_imagenet [ood]                  0.000004          0.000847   
tiny_imagenet tiny_imagenet [miscls]               0.002244          0.002899   
              tiny_imagenet [selective]            0.000442          0.000570   
              imagenet_a [ood]                     0.002370          0.002358   
              imagenet_r [ood]                     0.001728          0.003457   
              imagenet_o [ood]                     0.001961          0.003811   

                                         mahalanobis  \
ind_dataset   eval                                     
cifar10       cifar10 [miscls]              0.002597   
              cifar10 [selective]           0.000044   
              cifar100 [ood]                0.001507   
              svhn [ood]                    0.005441   
              tiny_imagenet [ood]           0.001056   
cifar100      cifar10 [ood]                 0.004189   
              cifar100 [miscls]             0.006341   
              cifar100 [selective]          0.001328   
              svhn [ood]                    0.033548   
              tiny_imagenet [ood]           0.005384   
tiny_imagenet tiny_imagenet [miscls]        0.003688   
              tiny_imagenet [selective]     0.005281   
              imagenet_a [ood]              0.009082   
              imagenet_r [ood]              0.007956   
              imagenet_o [ood]              0.004898   

                                         composite eat logscore outer outer + m  
ind_dataset   eval                                                               
cifar10       cifar10 [miscls]                                         0.001925  
              cifar10 [selective]                                      0.000100  
              cifar100 [ood]                                           0.000899  
              svhn [ood]                                               0.004980  
              tiny_imagenet [ood]                                      0.000620  
cifar100      cifar10 [ood]                                     

Pareto Percentages:
COMPOSITE EAT LOGSCORE OUTER OUTER + M 82.85714285714286
R_e 1 1 (Logscore) 0.0
R_t 1 1 (Logscore) 65.71428571428571
R_b 1 (Logscore) 21.904761904761905
mahalanobis 0.0


In [12]:
# assume your df has MultiIndex with level names ['ind_dataset', 'eval']
lvl = res_df_with_dominance.index.get_level_values('eval')

df_ood       = res_df_with_dominance[lvl.str.contains(r'\[ood\]')]
df_miscls    = res_df_with_dominance[lvl.str.contains(r'\[miscls\]')]
df_selective = res_df_with_dominance[lvl.str.contains(r'\[selective\]')]

# assume your df has MultiIndex with level names ['ind_dataset', 'eval']
lvl_std = res_df_std.index.get_level_values('eval')

df_ood_std       = res_df_std[lvl_std.str.contains(r'\[ood\]')]
df_miscls_std    = res_df_std[lvl_std.str.contains(r'\[miscls\]')]
df_selective_std = res_df_std[lvl_std.str.contains(r'\[selective\]')]

df_ood = df_ood.drop(columns=['if_dominates_100%', 'if_dominates_75%', 'if_dominates_50%',
       'beats_worst_component'])

df_miscls = df_miscls.drop(columns=['if_dominates_100%', 'if_dominates_75%', 'if_dominates_50%',
       'beats_worst_component'])

df_selective = df_selective.drop(columns=['if_dominates_100%', 'if_dominates_75%', 'if_dominates_50%',
    'beats_worst_component'])

In [13]:
col_order = ["composite eat logscore outer outer + m", "R_e 1 1 (Logscore)", "R_t 1 1 (Logscore)", "R_b 1 (Logscore)", "mahalanobis"]

In [14]:
df_ood = with_avg_row(df_ood, label=("AVG", "[all rows]"))[col_order]
df_ood_std = with_avg_row(df_ood_std, label=("AVG", "[all rows]"))[col_order]

latex_df = mean_pm_std(df_mean=df_ood, df_std=df_ood_std, )
latex_table = latex_df.to_latex(escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
 &  & composite eat logscore outer outer + m & R_e 1 1 (Logscore) & R_t 1 1 (Logscore) & R_b 1 (Logscore) & mahalanobis \\
ind_dataset & eval &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{cifar10} & cifar100 [ood] & $0.918 \pm 0.001$ & $0.905 \pm 0.000$ & $0.912 \pm 0.001$ & $0.917 \pm 0.001$ & $0.912 \pm 0.002$ \\
 & svhn [ood] & $0.957 \pm 0.005$ & $0.943 \pm 0.012$ & $0.957 \pm 0.007$ & $0.963 \pm 0.002$ & $0.934 \pm 0.005$ \\
 & tiny_imagenet [ood] & $0.912 \pm 0.001$ & $0.896 \pm 0.000$ & $0.904 \pm 0.001$ & $0.911 \pm 0.001$ & $0.910 \pm 0.001$ \\
\cline{1-7}
\multirow[t]{3}{*}{cifar100} & cifar10 [ood] & $0.765 \pm 0.001$ & $0.725 \pm 0.001$ & $0.774 \pm 0.002$ & $0.773 \pm 0.002$ & $0.535 \pm 0.004$ \\
 & svhn [ood] & $0.870 \pm 0.006$ & $0.756 \pm 0.013$ & $0.868 \pm 0.006$ & $0.870 \pm 0.007$ & $0.679 \pm 0.034$ \\
 & tiny_imagenet [ood] & $1.000 \pm 0.000$ & $1.000 \pm 0.000$ & $1.000 \pm 0.000$ & $0.810 \pm 0.001$ & $0.623 \pm 0.005$ \\
\cl

In [15]:
df_miscls = with_avg_row(df_miscls, label=("AVG", "[all rows]"))[col_order]
df_miscls_std = with_avg_row(df_miscls_std, label=("AVG", "[all rows]"))[col_order]

latex_df = fmt_valvar(df_miscls, df_miscls_std, mean_decimals=3, std_decimals=3, bold=False, underline=False)
latex_table = latex_df.to_latex(escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
 & measure & composite eat logscore outer outer + m & R_e 1 1 (Logscore) & R_t 1 1 (Logscore) & R_b 1 (Logscore) & mahalanobis \\
ind_dataset & eval &  &  &  &  &  \\
\midrule
cifar10 & cifar10 [miscls] & \valvar{0.944}{.002} & \valvar{0.940}{.003} & \valvar{0.943}{.002} & \valvar{0.942}{.002} & \valvar{0.928}{.003} \\
\cline{1-7}
cifar100 & cifar100 [miscls] & \valvar{0.849}{.003} & \valvar{0.818}{.003} & \valvar{0.853}{.003} & \valvar{0.845}{.003} & \valvar{0.574}{.006} \\
\cline{1-7}
tiny_imagenet & tiny_imagenet [miscls] & \valvar{0.847}{.002} & \valvar{0.813}{.001} & \valvar{0.851}{.002} & \valvar{0.845}{.003} & \valvar{0.417}{.004} \\
\cline{1-7}
AVG & [all rows] & \valvar{0.880}{.002} & \valvar{0.857}{.002} & \valvar{0.882}{.002} & \valvar{0.877}{.002} & \valvar{0.639}{.004} \\
\cline{1-7}
\bottomrule
\end{tabular}



In [16]:
df_selective = with_avg_row(df_selective, label=("AVG", "[all rows]"))[col_order]
df_selective_std = with_avg_row(df_selective_std, label=("AVG", "[all rows]"))[col_order]

latex_df = fmt_valvar(df_selective, df_selective_std, mean_decimals=3, std_decimals=3, bold=False, underline=False)
latex_table = latex_df.to_latex(escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
 & measure & composite eat logscore outer outer + m & R_e 1 1 (Logscore) & R_t 1 1 (Logscore) & R_b 1 (Logscore) & mahalanobis \\
ind_dataset & eval &  &  &  &  &  \\
\midrule
cifar10 & cifar10 [selective] & \valvar{0.997}{.000} & \valvar{0.997}{.000} & \valvar{0.997}{.000} & \valvar{0.997}{.000} & \valvar{0.996}{.000} \\
\cline{1-7}
cifar100 & cifar100 [selective] & \valvar{0.916}{.001} & \valvar{0.910}{.001} & \valvar{0.918}{.001} & \valvar{0.916}{.001} & \valvar{0.811}{.001} \\
\cline{1-7}
tiny_imagenet & tiny_imagenet [selective] & \valvar{0.886}{.001} & \valvar{0.879}{.001} & \valvar{0.891}{.000} & \valvar{0.889}{.001} & \valvar{0.660}{.005} \\
\cline{1-7}
AVG & [all rows] & \valvar{0.933}{.001} & \valvar{0.929}{.001} & \valvar{0.935}{.000} & \valvar{0.934}{.001} & \valvar{0.822}{.002} \\
\cline{1-7}
\bottomrule
\end{tabular}



In [17]:
import pandas as pd
import numpy as np

def highlight_best_and_second(s, best="#27ef56", second="#3908ed"):
    # rank 1 = largest; ties share the same rank
    r = s.rank(method="min", ascending=False)
    out = []
    for val, ri in zip(s, r):
        if pd.isna(val):
            out.append("")
        elif ri == 1:
            out.append(f"background-color: {best}; font-weight: bold")
        elif ri == 2:
            out.append(f"background-color: {second}")
        else:
            out.append("")
    return out


In [18]:
[el for el in transformed_df.columns]

['R_b 1 (Brier)',
 'R_b 1 (Logscore)',
 'R_b 1 (Spherical)',
 'R_b 1 (Zero-one)',
 'R_b 2 (Brier)',
 'R_b 2 (Logscore)',
 'R_b 2 (Spherical)',
 'R_b 2 (Zero-one)',
 'R_b 3 (Brier)',
 'R_b 3 (Logscore)',
 'R_b 3 (Spherical)',
 'R_b 3 (Zero-one)',
 'R_e 1 1 (Brier)',
 'R_e 1 1 (Logscore)',
 'R_e 1 1 (Spherical)',
 'R_e 1 1 (Zero-one)',
 'R_e 1 2 (Brier)',
 'R_e 1 2 (Logscore)',
 'R_e 1 2 (Spherical)',
 'R_e 1 2 (Zero-one)',
 'R_e 1 3 (Brier)',
 'R_e 1 3 (Logscore)',
 'R_e 1 3 (Spherical)',
 'R_e 1 3 (Zero-one)',
 'R_e 2 1 (Brier)',
 'R_e 2 1 (Logscore)',
 'R_e 2 1 (Spherical)',
 'R_e 2 1 (Zero-one)',
 'R_e 2 2 (Brier)',
 'R_e 2 2 (Logscore)',
 'R_e 2 2 (Spherical)',
 'R_e 2 2 (Zero-one)',
 'R_e 2 3 (Brier)',
 'R_e 2 3 (Logscore)',
 'R_e 2 3 (Spherical)',
 'R_e 2 3 (Zero-one)',
 'R_e 3 1 (Brier)',
 'R_e 3 1 (Logscore)',
 'R_e 3 1 (Spherical)',
 'R_e 3 1 (Zero-one)',
 'R_e 3 2 (Brier)',
 'R_e 3 2 (Logscore)',
 'R_e 3 2 (Spherical)',
 'R_e 3 2 (Zero-one)',
 'R_e 3 3 (Brier)',
 'R_e 3 3 (Log

In [30]:
# col_order = ['composite bayes all outer', 'R_b 1 (Logscore)', 'R_b 1 (Brier)', 'R_b 1 (Spherical)', 'R_b 1 (Zero-one)']
col_order = ['composite excess all outer outer', 'R_e 1 1 (Logscore)', 'R_e 1 1 (Brier)', 'R_e 1 1 (Spherical)', 'R_e 1 1 (Zero-one)']
transformed_df[col_order].style.apply(highlight_best_and_second, axis=1)

In [31]:
# col_order = ['composite bayes all inner', 'R_b 2 (Logscore)', 'R_b 2 (Brier)', 'R_b 2 (Spherical)', 'R_b 2 (Zero-one)']
col_order = ['composite excess all outer inner', 'R_e 1 2 (Logscore)', 'R_e 1 2 (Brier)', 'R_e 1 2 (Spherical)', 'R_e 1 2 (Zero-one)']
transformed_df[col_order].style.apply(highlight_best_and_second, axis=1)

In [32]:
# col_order = ['composite bayes all central', 'R_b 3 (Logscore)', 'R_b 3 (Brier)', 'R_b 3 (Spherical)', 'R_b 3 (Zero-one)']
col_order = ['composite excess all outer central', 'R_e 1 3 (Logscore)', 'R_e 1 3 (Brier)', 'R_e 1 3 (Spherical)', 'R_e 1 3 (Zero-one)']
transformed_df[col_order].style.apply(highlight_best_and_second, axis=1)

In [36]:
# col_order = ['composite bayes all outer', 'R_b 1 (Logscore)', 'R_b 1 (Brier)', 'R_b 1 (Spherical)', 'R_b 1 (Zero-one)']
col_order = ['composite excess all outer central', 'R_e 1 3 (Logscore)', 'R_e 1 3 (Brier)', 'R_e 1 3 (Spherical)', 'R_e 1 3 (Zero-one)']
latex_df = mean_pm_std(
    df_mean=with_avg_row(transformed_df[col_order], label=("AVG", "[all rows]"))[col_order],
    df_std=with_avg_row(std_df[col_order], label=("AVG", "[all rows]"))[col_order], )
latex_table = latex_df.to_latex(escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
 &  & composite excess all outer central & R_e 1 3 (Logscore) & R_e 1 3 (Brier) & R_e 1 3 (Spherical) & R_e 1 3 (Zero-one) \\
ind_dataset & eval &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{cifar10} & cifar10 [miscls] & $0.942 \pm 0.003$ & $0.936 \pm 0.003$ & $0.942 \pm 0.003$ & $0.942 \pm 0.003$ & $0.797 \pm 0.008$ \\
 & cifar10 [selective] & $0.997 \pm 0.000$ & $0.997 \pm 0.000$ & $0.997 \pm 0.000$ & $0.997 \pm 0.000$ & $0.983 \pm 0.002$ \\
 & cifar100 [ood] & $0.904 \pm 0.000$ & $0.903 \pm 0.001$ & $0.902 \pm 0.000$ & $0.904 \pm 0.000$ & $0.755 \pm 0.001$ \\
 & svhn [ood] & $0.941 \pm 0.011$ & $0.940 \pm 0.013$ & $0.940 \pm 0.010$ & $0.942 \pm 0.010$ & $0.825 \pm 0.038$ \\
 & tiny_imagenet [ood] & $0.895 \pm 0.000$ & $0.894 \pm 0.001$ & $0.893 \pm 0.000$ & $0.894 \pm 0.001$ & $0.752 \pm 0.002$ \\
\cline{1-7}
\multirow[t]{5}{*}{cifar100} & cifar10 [ood] & $0.710 \pm 0.002$ & $0.718 \pm 0.002$ & $0.681 \pm 0.002$ & $0.715 \pm 0.002$ & $0.689 \pm 0.00

In [24]:
transformed_df[col_order].style.apply(highlight_best_and_second, axis=1)

In [25]:
transformed_df[col_order].style.apply(highlight_best_and_second, axis=1)